In [2]:
import requests
from catsim.cat import generate_item_bank
from catsim.simulation import *
from catsim.initialization import *
from catsim.selection import *
from catsim.estimation import *
from catsim.stopping import *
import catsim.plot as catplot
from catsim.irt import icc
import matplotlib.pyplot as plt
from NODO import NODO
from RED_BAYESIANA import RED_BAYESIANA
from MPCN import MPCN
import pickle
import os.path
from os import path
import pyAgrum as gum
import random
from PRIMER_TEMA import PRIMER_TEMA



URL = "http://159.223.190.216/arbol/caminoslibreria"
respuestaDelServidor = requests.get(url = URL).json()
caminos = respuestaDelServidor['caminos']
CONTRUCTOR_BN = RED_BAYESIANA()

primer_tema = PRIMER_TEMA("http://159.223.190.216/arbol/caminoslibreria")
tema_elegido = primer_tema.calcularPrimerTema()

###### Información a ser pasada al módulo #####
id_alumno = 20
saltos = 1
tema_objetivo = 80
ponderacion_objetiva = 70
##############################################

nombre_rbma = "rbma_xml/rbma_alumno_"+str(id_alumno)+".bifxml"
nombre_rbma_pickle = "rbma_pickle/rbma_alumno_"+str(id_alumno)+".pkl" #pickle
nombre_rmb = "rbm/rbm_1.bifxml"
CONTRUCTOR_BN.configurarRBM(nombre_rmb) # Asignamos el nombre de la Red Bayesiana Maestra a utilizar

RBMA = {}
RBMA_XML = ""
if not path.exists(nombre_rbma_pickle):
    RBMA_XML = CONTRUCTOR_BN.obtenerRedCompleta() # se realiza una copia de la RBM


    for nodo in RBMA_XML.nodes():
        nodo_rbm = NODO(RBMA_XML.variable(nodo).name(), 1)
        for tema in respuestaDelServidor['temas']:
            if RBMA_XML.variable(nodo).name() == str(tema['id']):
                nodo_rbm.dificultad = tema['dificultad']
        RBMA[RBMA_XML.variable(nodo).name()] = nodo_rbm

    # Almacenar la RBMA de manera persistente
    with open(nombre_rbma_pickle, 'wb') as outp:
        pickle.dump( RBMA, outp, pickle.HIGHEST_PROTOCOL)
    gum.saveBN(RBMA_XML, nombre_rbma)
else:
    with open(nombre_rbma_pickle, 'rb') as inp:
        RBMA = pickle.load(inp)
    RBMA_XML = gum.loadBN(nombre_rbma)

CONTRUCTOR_BN.configurarParametros([tema_objetivo,ponderacion_objetiva,saltos])
SRB = CONTRUCTOR_BN.obtenerSubRed() # 508


CAMINOS_SRB = CONTRUCTOR_BN.obtenerCaminosSubRed(tema_objetivo, saltos, caminos)

INSTANCIA_MPCN = MPCN(RBMA, CAMINOS_SRB, tema_objetivo, ponderacion_objetiva)
INSTANCIA_MPCN.ponderarNodos(SRB) #
INSTANCIA_MPCN.hacerInferencias(RBMA_XML)
INSTANCIA_MPCN.promediarInferencias(SRB)

RBMA = INSTANCIA_MPCN.obtenerNodos()  #Se obtiene la RBMA actualizada con las inferenicas promediadas de la SRB
items = INSTANCIA_MPCN.obtenerItems() #Calcula las caracteristicas de Seudoadivinacion, Discriminzacion y Dificultad

In [3]:
items

array([[ 1.        , -1.97142857,  0.26361311,  1.        ,  0.        ],
       [ 1.        , -0.25714286,  0.18770847,  1.        ,  0.        ],
       [ 1.        , -0.25714286,  0.23233243,  1.        ,  0.        ],
       [ 1.        , -0.25714286,  0.32894743,  1.        ,  0.        ],
       [ 1.        , -2.05714286,  0.31066726,  1.        ,  0.        ],
       [ 1.        , -2.14285714,  0.38119077,  1.        ,  0.        ],
       [ 1.        , -2.05714286,  0.12050168,  1.        ,  0.        ],
       [ 1.        , -1.11428571,  0.41935667,  1.        ,  0.        ],
       [ 1.        , -0.34285714,  0.25162089,  1.        ,  0.        ],
       [ 1.        , -2.14285714,  0.3996576 ,  1.        ,  0.        ],
       [ 1.        , -2.22857143,  0.24280329,  1.        ,  0.        ],
       [ 1.        , -2.31428571,  0.2617665 ,  1.        ,  0.        ],
       [ 1.        , -1.2       ,  0.32244679,  1.        ,  0.        ],
       [ 1.        , -0.42857143,  0.2

In [3]:
temas_evaluados = []
temas_evaluados_ids = []
respuestas = []
for nodo in RBMA:
    if RBMA[nodo].clasificacion == 3:
        temas_evaluados.append(0)
        temas_evaluados_ids.append(0)
        respuestas.append(0)
i=0
for nodo in RBMA:
    if RBMA[nodo].clasificacion == 3:
        temas_evaluados_ids[RBMA[nodo].orden_evaluacion] =  RBMA[nodo].id
        temas_evaluados[RBMA[nodo].orden_evaluacion] = i
        if RBMA[nodo].ponderacion > 70:
            respuestas[RBMA[nodo].orden_evaluacion] = True
        else:
            respuestas[RBMA[nodo].orden_evaluacion] = False
    i=i+1

In [6]:
respuestas

[True]

In [7]:
temas_evaluados = [57]
respuestas = [True]
initializer = RandomInitializer()
selector = MaxInfoSelector()
estimator = NumericalSearchEstimator()
#stopper = MaxItemStopper(20) #
stopper = MinErrorStopper(0.3) # Error estandar 0.3. Entre menos valor, más precision en la estimación

est_theta = 0.0

primera_evluacion = True
for i in range(100):
    print(i)
    if primera_evluacion:
        new_theta = estimator.estimate(items=items, administered_items=temas_evaluados, response_vector=respuestas, est_theta=est_theta)
        primera_evluacion = False
    else:
        new_theta = estimator.estimate(items=items, administered_items=temas_evaluados, response_vector=respuestas, est_theta=new_theta)

    _stop = stopper.stop(administered_items=items[temas_evaluados], theta=new_theta)

    sig_tema = selector.select(items=items, administered_items=temas_evaluados, est_theta=new_theta)

    temas_evaluados.append(sig_tema)
    respuestas.append(random.choice([True, False]))
    print(temas_evaluados)
    print(respuestas)
    print("Debe detenerse la evaluación? ", _stop)
    print("Theta prima en iteracion", new_theta)
    print("--------------------------")


0
[57, 21]
[True, False]
Debe detenerse la evaluación?  False
Theta prima en iteracion 1.5
--------------------------
1
[57, 21, 1]
[True, False, False]
Debe detenerse la evaluación?  False
Theta prima en iteracion 0.08571428498971273
--------------------------
2
[57, 21, 1, 98]
[True, False, False, False]
Debe detenerse la evaluación?  False
Theta prima en iteracion -1.0334469564475535
--------------------------
3
[57, 21, 1, 98, 6]
[True, False, False, False, True]
Debe detenerse la evaluación?  False
Theta prima en iteracion -1.7046367831190015
--------------------------
4
[57, 21, 1, 98, 6, 84]
[True, False, False, False, True, False]
Debe detenerse la evaluación?  False
Theta prima en iteracion -1.315340498155477
--------------------------
5
[57, 21, 1, 98, 6, 84, 58]
[True, False, False, False, True, False, False]
Debe detenerse la evaluación?  False
Theta prima en iteracion -1.7637085556282113
--------------------------
6
[57, 21, 1, 98, 6, 84, 58, 89]
[True, False, False, False

In [ ]:
print("NODOS EVALUADOS ==================================")
for evaluado in temas_evaluados:
    print("ID NODO: " + str( RBMA[ list(RBMA)[evaluado] ].id ) )
    print("DIFICULTAD: " + str( RBMA[ list(RBMA)[evaluado] ].dificultad ) )
    print("RESPUESTA: " + str(respuestas[ temas_evaluados.index(evaluado) ]))
    print("_____________________")
print("NODO SIGUIENTE ==================================")
print("ID NODO: " + str(  RBMA[ list(RBMA)[sig_tema] ].id )  )
print("DIFICULTAD: " + str(  RBMA[ list(RBMA)[sig_tema] ].dificultad )  )

with open(nombre_rbma_pickle, 'wb') as outp:
    pickle.dump( RBMA, outp, pickle.HIGHEST_PROTOCOL)

gum.saveBN(RBMA_XML, nombre_rbma)